For the ETL process the bank market capitalization and the currency exchange rates are needed.
1. The bank market capitalization is extracted through **web scraping** from Wikipedia [https://en.wikipedia.org](https://en.wikipedia.org/wiki/List_of_largest_banks).
2. The currency exchange rates are imported using the **ExchangeRate-API** [https://exchangeratesapi.io/](https://exchangeratesapi.io/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01).

Importing the requiered libraries:

In [1]:
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup
import html5lib
import requests
import pandas as pd

## Web Scraping for the bank market capitalization

In [2]:
# getting the data from wikipedia
html_data = requests.get('https://en.wikipedia.org/wiki/List_of_largest_banks').text
soup = BeautifulSoup(html_data, 'html.parser')

largest_banks = pd.DataFrame(columns=['Bank name', 'Market cap (US billion)'])

# extracting the necessary data from each row and appending it to the empty dataframe
for row in soup.find_all('tbody')[3].find_all('tr'): 
    col = row.find_all('td')

    if (col != []):
        Name = col[1].text.replace('\n','')
        Market_cap = col[2].text.replace('\n','')
        largest_banks = largest_banks.append({'Bank name':Name, 'Market cap (US billion)':Market_cap}, ignore_index=True)

# cleaning and changing data type
largest_banks['Market cap (US billion)'].replace('\[.\]', '', regex=True, inplace = True)  
largest_banks = largest_banks.astype({'Market cap (US billion)': float})

# loading the data as a JSON file
largest_banks.to_json('bank_market_cap.json')

Alternative, pandas has a **read_html** methode, which can make the job easier.

In [3]:
# html = pd.read_html('https://en.wikipedia.org/wiki/List_of_largest_banks')

# largest_banks = pd.DataFrame(html[3])
# largest_banks.rename(columns={'Market cap(US$ billion)': 'Market cap (US billion)'}, inplace = True)
# largest_banks = largest_banks[['Bank name', 'Market cap (US billion)']]
# largest_banks['Market cap (US billion)'].replace('\[.\]', '', regex=True, inplace = True)
# largest_banks = largest_banks.astype({'Market cap (US billion)': float}) 
# largest_banks
# largest_banks.to_json('bank_market_cap.json')

## API for the currency exchange rates

Using ExchangeRate-API the currency exchange rate data can be imported. The steps below show how to get the access key to get the exchange rates.

1.  Open the url : [https://exchangeratesapi.io/](https://exchangeratesapi.io/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01) and click on **Get Free API Key**.
2.  Subscribe for Free plan and Sign-in.
3.  Once the account is created you will be redirected to [https://apilayer.com](https://apilayer.com/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork23455645-2022-01-01) website.
4.  Now, click on the **user icon** and click **Account**.

3.  Scroll down and you will get the API Key section. Copy the API key and use in the url.



In [4]:
# url = '***' 
# output = requests.get(url).text
# exchange_rates = pd.read_json(output)
# exchange_rates = exchange_rates[['rates']]
# exchange_rates.to_csv('exchange_rates.csv')

As the exchange rate fluctuates, for the ETL process the exchange rates will be extracted from the repository (from July 22nd, 2022) to make it simpler.


# Extract


### JSON extract function

This function will extract the JSON file.


In [5]:
def extract_from_json(json_file):
    dataframe = pd.read_json(json_file)
    return dataframe

### CSV extract function

This function will extract the CSV file.


In [6]:
def extract_from_csv(csv_file, currency):
    exchange_rate = round(pd.read_csv(csv_file, index_col = 0).loc[currency][0],4)
    return exchange_rate

# Transform

Using any <code>exchange_rate</code> the `Market cap (US Billion)` column  from the `bank_market_cap.json` file will be changed from USD to the choosen currancy.


In [7]:
def transform(DataFrame, exhange_rate, currency):
    DataFrame.iloc[:,1]= round(DataFrame.iloc[:,1]*exchange_rate, 3)
    DataFrame.rename(columns={'Market cap (US billion)': 'Market cap (%s billion)'%(currency)}, inplace = True)
    return DataFrame

# Load

The load function takes a dataframe and load it to a new json file including the name of the choosen currency.

In [8]:
def load(DataFrame, currency):
    DataFrame.to_json('bank_market_cap_%s.json'%(currency))

## Logging function


In [9]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format)
    with open('logfile.txt','a') as f:
        f.write(timestamp + ',' + message + '\n')

## Running the ETL process


In [10]:
# enter any currency

currency = 'EUR'

In [11]:
# run the ETL Process

log('ETL process started for %s'%(currency))

log('Extract bank market capitalization started')
bank_market_cap = extract_from_json('bank_market_cap.json')
log('Extract ended')

log('Extract the exchange rate started')
exchange_rate = extract_from_csv('exchange_rates.csv', currency)
log('Extract ended')

log('Transformation started')
transformed_market = transform(bank_market_cap, exchange_rate, currency)
log('Transformation ended')

log('Load started')
load(transformed_market, currency)
log('Load ended')
log('_____________ETL process ended________________')

This ETL Process is an upgraded version of an assignment in the IBM Data Engineering Certificate.